#Прогнозирование стоимости медицинской страховки с использованием линейной регрессии
В этом ноутбуке проводится анализ данных о страховых взносах с использованием различных техник предобработки данных и построения моделей линейной регрессии для предсказания логарифма стоимости страховки. Мы исследуем влияние различных признаков на стоимость страховых взносов, обучая модель линейной регрессии, используя стандартизацию и полиномиальные признаки.

## Содержание

1. **Импорт необходимых библиотек**
2. **Загрузка и подготовка данных**
3. **Описание функций**

 3.1. **Функция распределения данных**

 3.2. **Функция для оценки модели регрессии**

 3.3. **Функция для вывод результатов модели**
4. **Обучение модели на масштабированных данных**
5. **Использование всех признаков (включая качественные)**
6. **Применение полиномиальных признаков**
7. **Заключение**


### 1. Импорт необходимых библиотек

В этом блоке импортируются необходимые библиотеки для работы с данными и моделями машинного обучения.

In [ ]:
from math import sqrt
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures

- **math** и **numpy**: для выполнения математических операций и работы с массивами данных.
- **pandas**: для работы с таблицами данных.
- **matplotlib**: для визуализации данных.
- **sklearn.linear_model.LinearRegression**: для применения линейной регрессии.
- **sklearn.metrics**: для вычисления метрик модели.
- **sklearn.model_selection.train_test_split**: для разделения данных на обучающую и тестовую выборки.
- **sklearn.preprocessing.MinMaxScaler** и **PolynomialFeatures**: для масштабирования данных и работы с полиномиальными признаками.

### 2. Загрузка и подготовка данных

В этом блоке загружается датасет и производится подготовка данных для анализа.

In [ ]:
from google.colab import files
uploaded = files.upload()
data = pd.read_csv('insurance.csv')
data['log_charges'] = np.log1p(data['charges'])
data = data.drop('charges', axis=1)

quantitative = ['age', 'bmi', 'children']  # количественные
qualitative = ['sex', 'smoker', 'region']  # качественные

scaled = MinMaxScaler().fit_transform(data[quantitative])
scaled_df = pd.DataFrame(scaled, columns=quantitative)

Saving insurance.csv to insurance.csv


- Датасет `insurance.csv` загружается с помощью **pandas**.
- Для улучшения нормализации данных добавляется новый столбец `log_charges`, который является логарифмом от оригинальной переменной `charges`.
- Данные разделяются на количественные и качественные признаки.
- Количественные признаки масштабируются с помощью **MinMaxScaler**, который приводит значения в диапазон от 0 до 1.

### 3. Описание функций

#### 3.1. Функция распределения данных

Эта функция используется для визуализации распределения переменной `charges` и её логарифмического преобразования.

In [ ]:
def distribution(old, new):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(6, 3))
    axes[0].hist(old, bins=15, color='skyblue', edgecolor='black')
    axes[0].set_xlabel('Charges')
    axes[0].set_ylabel('Frequency')
    axes[1].hist(new, bins=15, color='salmon', edgecolor='black')
    axes[1].set_xlabel('Log Charges')
    axes[1].set_ylabel('Frequency')
    plt.tight_layout()
    plt.show()

Эта функция строит два гистограммных графика: один для оригинальных значений `charges`, второй — для их логарифмического преобразования.

#### 3.2. Функция для оценки модели регрессии

Функция `evaluate_regression` оценивает качество модели на основе нескольких метрик: MSE, R² и RSD.

In [ ]:
def evaluate_regression(y_train, y_train_pred, y_test, y_test_pred):
    mse_train = mean_squared_error(y_train, y_train_pred)
    mse_test = mean_squared_error(y_test, y_test_pred)

    r2_train = r2_score(y_train, y_train_pred) * 100
    r2_test = r2_score(y_test, y_test_pred) * 100

    rsd_train = (sqrt(mse_train) / np.mean(y_train)) * 100
    rsd_test = (sqrt(mse_test) / np.mean(y_test)) * 100

    output(mse_train, mse_test, rsd_train, rsd_test, r2_train, r2_test)

- **mean_squared_error**: вычисляет среднюю квадратическую ошибку.
- **r2_score**: вычисляет коэффициент детерминации (R²), который показывает, как хорошо модель объясняет данные.
- **sqrt(MSE)**: используется для расчета средней относительной погрешности (RSD).

#### 3.3. Функция для обучение модели на различных признаках

Функция `learning` обучает модель линейной регрессии на различных комбинациях признаков.

In [ ]:
def learning(X, Y):
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    evaluate_regression(y_train, y_train_pred, y_test, y_test_pred)

- Данные разделяются на обучающую и тестовую выборки.
- Модель линейной регрессии обучается на обучающих данных и оценивается на тестовых.

#### 3.4. Функция для вывод результатов модели

Функция `output` выводит метрики модели в виде таблицы для обучающих и тестовых данных.

In [ ]:
def output(mse_train, mse_test, rsd_train, rsd_test, r2_train, r2_test):
    metrics_dict = {'Средняя квадратическая ошибка (MSE)': [f"{mse_train:.3f}", f"{mse_test:.3f}"],
                    'Средняя относительная погрешность (RSD)': [f"{rsd_train:.3f}%", f"{rsd_test:.3f}%"],
                    'Коэффициент детерминации (R^2)': [f"{r2_train:.3f}%", f"{r2_test:.3f}%"]}
    metrics_df = pd.DataFrame.from_dict(metrics_dict, orient='index',
                                        columns=['ОБУЧАЮЩАЯ', 'ТЕСТОВАЯ'])
    print(metrics_df)

### 4. Обучение модели на масштабированных данных

In [ ]:
print("Только количественные признаки")
learning(scaled_df, data['log_charges'])

Только количественные признаки
                                        ОБУЧАЮЩАЯ ТЕСТОВАЯ
Средняя квадратическая ошибка (MSE)         0.561    0.654
Средняя относительная погрешность (RSD)    8.234%   8.885%
Коэффициент детерминации (R^2)            30.931%  28.872%


### 5. Использование всех признаков (включая качественные)

Применим кодирование категориальных признаков с помощью `get_dummies` для качественных признаков и обучим модель на всех признаках.

In [ ]:
data_encoded = pd.get_dummies(data[qualitative], columns=qualitative, drop_first=True)
data_df = pd.concat([data_encoded, scaled_df], axis=1)
print("\nОт всех признаков, включая качественные")
learning(data_df, data['log_charges'])


От всех признаков, включая качественные
                                        ОБУЧАЮЩАЯ ТЕСТОВАЯ
Средняя квадратическая ошибка (MSE)         0.213    0.161
Средняя относительная погрешность (RSD)    5.048%   4.459%
Коэффициент детерминации (R^2)            74.769%  80.550%


### 6. Применение полиномиальных признаков

Для улучшения модели можно использовать полиномиальные признаки. Это позволяет выявить нелинейные зависимости в данных.


In [ ]:
plf = PolynomialFeatures(degree=2, include_bias=False)
X_poly = plf.fit_transform(scaled_df)
X_poly_df = pd.DataFrame(X_poly, columns=plf.get_feature_names_out(quantitative))
X_poly_encoded = pd.concat([X_poly_df, data_df], axis=1)
print("Полином")
learning(X_poly_encoded, data['log_charges'])

Полином
                                        ОБУЧАЮЩАЯ ТЕСТОВАЯ
Средняя квадратическая ошибка (MSE)         0.195    0.180
Средняя относительная погрешность (RSD)    4.846%   4.673%
Коэффициент детерминации (R^2)            76.927%  78.708%


Здесь формируется словарь с метриками, который затем преобразуется в **DataFrame** для удобного отображения.

### 7. Заключение

Этот блокнот демонстрирует, как использовать линейную регрессию для предсказания стоимости медицинской страховки на основе различных признаков. Модели оцениваются с помощью различных метрик, и результаты выводятся в удобном формате для анализа.